pip install dash==1.0.2  # The core dash backend <br>
pip install dash-daq==0.1.0  # DAQ components (newly open-sourced!)

Начиная с какой то версии dash автоматически инсталлит базовые зависимости, больше нет нужды перечилять их все

In [ ]:
!pip install dash

In [ ]:
!pip install dash-daq

In [1]:
from dash import dcc
print(dcc.__version__)

2.13.0


In [ ]:
!pip install pyngrok

dash app состоит из 2 компонент
 - layout - расположение элементов, древовидная структура отображаемых элементов, в общем все то что мы видим
 - callback - реактовая компонента, отвечает за взаимодейсвтие

In [5]:
import pandas as pd
pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv').head()

,country,pop,continent,lifeExp,gdpPercap
0,Afghanistan,31889923.0,Asia,43.828,974.580338
1,Albania,3600523.0,Europe,76.423,5937.029526
2,Algeria,33333216.0,Africa,72.301,6223.367465
3,Angola,12420476.0,Africa,42.731,4797.231267
4,Argentina,40301927.0,Americas,75.320,12779.379640


In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# Incorporate data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

# Initialize the app
app = Dash()

# App layout
app.layout =  html.Div([
    html.Div(children='My First App with Data, Graph, and Controls'),
    html.Hr(),
    dcc.RadioItems(options=['pop', 'lifeExp', 'gdpPercap'], value='lifeExp', id='controls-and-radio-item'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph')
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import dash
from dash.dependencies import Input, Output
from dash import dash_table
from dash import dcc
from dash import html
import pandas as pd
from dash.dash_table.Format import Group

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='field-dropdown',
        options=[{'label': continent, 'value': continent} for continent in df['continent'].unique()],
        multi=True,
        value='Asia',
        clearable=False
    ),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="multi",
        row_deletable=True,
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Div(id='datatable-interactivity-container')
])

@app.callback(Output('datatable-interactivity', 'data'), [Input('field-dropdown', 'value')])
def update_table(user_selection):
    """
    For user selections, return the relevant table
    """
    if isinstance(user_selection,str):
        df_to_ret = df[df['continent'] == user_selection]
    else:
        df_to_ret = df[df['continent'].isin(user_selection)]
    return df_to_ret.to_dict('records')

@app.callback(
    Output('datatable-interactivity-container', "children"),
    [Input('datatable-interactivity', "derived_virtual_data"),
     Input('datatable-interactivity', "derived_virtual_selected_rows")])
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncracy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["country"],
                        "y": dff[column],
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["pop", "lifeExp", "gdpPercap"] if column in dff
    ]


if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import datetime

import dash
from dash import dcc
from dash import html
import plotly
from dash.dependencies import Input, Output

# pip install pyorbital
from pyorbital.orbital import Orbital
satellite = Orbital('TERRA')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    html.Div([
        html.H4('TERRA Satellite Live Feed'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='live-update-graph'),
# этот элемент отвечает за обновление дэшборда
# в данном случае будет обновление раз в 1000 milliseconds        
        dcc.Interval(
            id='interval-component',
            interval=1*1000, # in milliseconds
            n_intervals=0
        )
    ])
)


@app.callback(Output('live-update-text', 'children'),
              [Input('interval-component', 'n_intervals')])
def update_metrics(n):
    lon, lat, alt = satellite.get_lonlatalt(datetime.datetime.now())
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.Span('Longitude: {0:.2f}'.format(lon), style=style),
        html.Span('Latitude: {0:.2f}'.format(lat), style=style),
        html.Span('Altitude: {0:0.2f}'.format(alt), style=style)
    ]


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_graph_live(n):
    satellite = Orbital('TERRA')
    data = {
        'time': [],
        'Latitude': [],
        'Longitude': [],
        'Altitude': []
    }

    # Collect some data
    for i in range(180):
        time = datetime.datetime.now() - datetime.timedelta(seconds=i*20)
        lon, lat, alt = satellite.get_lonlatalt(
            time
        )
        data['Longitude'].append(lon)
        data['Latitude'].append(lat)
        data['Altitude'].append(alt)
        data['time'].append(time)

    # Create the graph with subplots
    fig = plotly.tools.make_subplots(rows=2, cols=1, vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    fig.append_trace({
        'x': data['time'],
        'y': data['Altitude'],
        'name': 'Altitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 1, 1)
    fig.append_trace({
        'x': data['Longitude'],
        'y': data['Latitude'],
        'text': data['time'],
        'name': 'Longitude vs Latitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 2, 1)

    return fig

server = app.server

if __name__ == '__main__':
    server.run(
        ssl_context=('cert.pem', 'key.pem'),
        host='0.0.0.0',
        port='8080'
        )


In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.datasets import load_iris, load_wine

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


############ Loading Datasets ##################
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["FlowerType"] = [iris.target_names[target] for target in iris.target]

wine = load_wine()

wine_df = pd.DataFrame(data=wine.data, columns=wine.feature_names)
wine_df["WineType"] = [wine.target_names[target] for target in wine.target]


apple_df = pd.read_csv("~/datasets/AAPL.csv")


################# Line Chart ##############################
chart1 = go.Figure()

chart1.add_trace(go.Scatter(x=apple_df.Date, y=apple_df.Open,
                            marker={"color":"tomato"},
                            mode="lines"))


chart1.update_layout(height=500,
                     xaxis_title="Date",
                     yaxis_title="Price ($)",
                     title="Apple Stock Prices [Apr-2019-Mar-2020]")

################# Scatter Plot ################################
chart2 = px.scatter(data_frame=wine_df,
                   x=wine.feature_names[0],
                   y=wine.feature_names[1],
                   color="WineType",
                   title="alcohol vs malic_acid color-encoded by wine type",
                   height=500,
                   )


################## Bar Chart ###############################                   
iris_avg_by_flower_type = iris_df.groupby(by="FlowerType").mean().reset_index()

chart3 = px.bar(data_frame=iris_avg_by_flower_type,
               x="FlowerType",
               y=iris.feature_names[0],
               title="Avg %s Per Flower Type"%iris.feature_names[0],
               height=500,
               )

#################### Creating App Object ############################               
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

####################### Setting Graphs as HTML Children ##############             
graph1 = dcc.Graph(
        id='graph1',
        figure=chart1,
        #className="eight columns"
    )

graph2 = dcc.Graph(
        id='graph2',
        figure=chart2,
        #className="five columns"
    )

graph3 = dcc.Graph(
        id='graph3',
        figure=chart3,
        #className="five columns"
    )


############### Creating Widgets For Each Graph #########################    
multi_select_line_chart = dcc.Dropdown(
        id="multi_select_line_chart",
        options=[{"value":label, "label":label} for label in ["Open", "Low", "High", "Close"]],
        value=["Open"],
        multi=True,
        clearable = False
    )

dropdown1_scatter_chart = dcc.Dropdown(
        id="dropdown1_scatter_chart",
        options=[{"value":label, "label":label} for label in wine.feature_names],
        value=wine.feature_names[0],
        className="six columns",
        clearable = False
    )

dropdown2_scatter_chart = dcc.Dropdown(
        id="dropdown2_scatter_chart",
        options=[{"value":label, "label":label} for label in wine.feature_names],
        value=wine.feature_names[1],
        className="six columns",
        clearable = False
    )

dropdown_bar_chart = dcc.Dropdown(
        id="dropdown_bar_chart",
        options=[{"value":label, "label":label} for label in iris.feature_names],
        value=iris.feature_names[0],
        clearable = False
    )


######################### Laying out Charts & Widgets to Create App Layout ##########
header = html.H2(children="Simple Dashboard With Widgets")

row1 = html.Div(children=[multi_select_line_chart, graph1], className="eight columns")

scatter_div = html.Div(children=[html.Div(children=[dropdown1_scatter_chart, dropdown2_scatter_chart], className="row") , graph2], className="six columns")

bar_div = html.Div(children=[dropdown_bar_chart, graph3], className="six columns")

row2 = html.Div(children=[scatter_div, bar_div], className="eight columns")

layout = html.Div(children=[header, row1, row2], style={"text-align": "center", "justifyContent":"center"})

############### Setting App Layout ########################################
app.layout = layout


################## Creating Callbacks for Each Widget ############################
@app.callback(Output('graph1', 'figure'), [Input('multi_select_line_chart', 'value')])
def update_line(price_options):
    chart1 = go.Figure()

    for price_op in price_options:
        chart1.add_trace(go.Scatter(x=apple_df.Date, y=apple_df[price_op],
                                mode="lines", name=price_op))


    chart1.update_layout(
                         xaxis_title="Date",
                         yaxis_title="Price ($)",
                         title="Apple Stock Prices [Apr-2019-Mar-2020]",
                         height=500,)
    return chart1



@app.callback(Output('graph2', 'figure'), [Input('dropdown1_scatter_chart', 'value'), Input('dropdown2_scatter_chart', 'value')])
def update_scatter(drop1, drop2):
    chart2 = px.scatter(data_frame=wine_df,
                   x=drop1,
                   y=drop2,
                   color="WineType",
                   title="%s vs %s color-encoded by wine type"%(drop1, drop2),
                   height=500,
                   )

    return chart2


@app.callback(Output('graph3', 'figure'), [Input('dropdown_bar_chart', 'value')])
def update_bar(bar_drop):
    chart3 = px.bar(data_frame=iris_avg_by_flower_type,
               x="FlowerType",
               y=bar_drop,
               title="Avg %s Per Flower Type"%bar_drop,
               height=500,
               )
    return chart3


################## Running App #####################################

if __name__ == "__main__":
    app.run_server(debug=True)